# Community Hydrological Model support on I-GUIDE platform - WRFHydro/National Water Model

The [I-GUIDE](https://iguide.illinois.edu/) platform integrates various state-of-the-art cyberinfrastructure (CI) capabilities to support Community Hydrological Modelling. [WRFHydro](https://ral.ucar.edu/projects/wrf_hydro), serving as the code base for the [NOAA National Water Model](https://water.noaa.gov/about/nwm) (NWM), is a leading-edge, open-source community hydrometeorological and hydrologic modelling system developed by [NCAR](https://ncar.ucar.edu/). 

This notebook demostrates the setup for a typical WRFHydro model on I-GUIDE platform leveraging different tools or services through out the entire end-to-end modelling workflow. 

In [ ]:
!pip install git+https://github.com/cybergis/cybergis-compute-python-sdk.git@v2

In [ ]:
import os
import json
import time
from datetime import datetime, timedelta
import shutil

import requests

## Setup Simulation Parameters

Pick a HUC-12 watershed, and specify StartDate and EndDate for simulation.
Notice: the simulation stops at 00:00 (12AM) on the EndDate.

In [ ]:
# huc12 id
huc12 = "070200030503"

# Start at 00:00 (12AM)
start_datetime = datetime(2016, 1, 1)
#                          Y   M   D
# End at 00:00 (12AM)
end_datetime = datetime(2016, 1, 7)
#                        Y   M   D

# version WRFHydro codebase on github (tag/release/commit id)
wrfhydro_version = "v5.2.0"

In [ ]:
params_subset_domain = {"huc12_id": huc12, 
                        "start_date": start_datetime.strftime("%m/%d/%Y"), 
                        "end_date": end_datetime.strftime("%m/%d/%Y")}
params_subset_domain

## Subset DOMAIN Files with GeoEDF Data Connector on CyberGIS Compute

The source of WRFHydro DOMAIN files is [CUAHSI Domain Subsetter](https://subset.cuahsi.org/) service. I-GUIDE provides a reusable [GeoEDF Data Connector](https://dl.acm.org/doi/10.1145/3311790.3396631) ([CUAHSISubsetterInput-Connector](https://github.com/I-GUIDE/cybergis-compute-cuahsisubsetterinput-connector) ) that makes requests to CUAHSI Domain Subsetter REST APIs and retrieves the domain files ready for model use. The GeoEDF Data Connector has been integrated into [CyberGIS-Compute](https://cybergis.github.io/cybergis-compute-python-sdk/reference.html) as a job that can be invoked by users from Jupyter environment and executed on supported HPC resources. The subset domain files staged remotely is ready for use by WRFHydro model on HPC, and user has the option to download the files from HPC back to Jupyter for local manipulation.

In [ ]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.show_ui(defaultJob="CUAHSI_Subsetter_Connector", input_params=params_subset_domain)

Retain Domain Subsetter JobID for later reference

In [ ]:
job_cuahsi_subset_domain = cybergis.job
domain_path_HPC = "{}".format(job_cuahsi_subset_domain.id)
domain_path_HPC

## Subset FORCINGS with GeoEDF Data Processor on CyberGIS Compute

The source of WRFHydro FORCING files is [AORC](https://registry.opendata.aws/nwm-archive/) dataset hosted on AWS. I-GUIDE provides a reusable [GeoEDF Data Processor](https://dl.acm.org/doi/10.1145/3311790.3396631) ([SubsetForcingData-Processor](https://github.com/I-GUIDE/cybergis-compute-subsetaorcforcingdata-processor) ) that subsets forcing files spatially and temporally. This GeoEDF Data Processor has been integrated into [CyberGIS-Compute](https://cybergis.github.io/cybergis-compute-python-sdk/reference.html) as a job that can be invoked by users from Jupyter environment and executed on supported HPC resources. The subset forcing files staged remotely is ready for use by WRFHydro model on HPC, and user has the option to download the files from HPC back to Jupyter for local manipulation.

In [ ]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.show_ui(defaultJob="Subset_AORC_Forcing_Data_Processor", input_params=params_subset_domain)

Retain Forcing Processor JobID for later reference

In [ ]:
job_subset_forcing = cybergis.job
forcing_path_HPC = "{}".format(job_subset_forcing.id)
forcing_path_HPC

## Prepare Model Configurations

In [ ]:
# setEnvar.sh ---> SPATIAL_SOIL=1 export HYDRO_D=1

In [ ]:
# Create a "Simulation" directory

workspace = os.getcwd()

simulation_dir = os.path.join(workspace, 'Simulation')
if os.path.exists(simulation_dir):
    shutil.rmtree(simulation_dir)
os.makedirs(simulation_dir)

#List of files
os.listdir(simulation_dir)

In [ ]:
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/{wrfhydro_version}/trunk/NDHMS/template/setEnvar.sh
! sed -i '/export HYDRO_D=0/c\export HYDRO_D=1' ./setEnvar.sh
! sed -i '/export SPATIAL_SOIL=0/c\export SPATIAL_SOIL=1' ./setEnvar.sh
! cat ./setEnvar.sh | grep -E 'HYDRO_D|SPATIAL_SOIL'
! mv ./setEnvar.sh {simulation_dir}

In [ ]:
# namelist.hrldas --> START_YEAR START_MONTH START_DAY START_HOUR START_MIN RESTART_FILENAME_REQUESTED
start_year = start_datetime.year
start_month = "{:02d}".format(start_datetime.month)
start_day = "{:02d}".format(start_datetime.day)
start_hour = "{:02d}".format(start_datetime.hour)
start_minute = "{:02d}".format(start_datetime.minute)
khour = (end_datetime - start_datetime) / timedelta(hours=1)
khour = "{}".format(int(khour))

In [ ]:
! rm -rf namelist.hrldas
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/{wrfhydro_version}/trunk/NDHMS/template/NoahMP/namelist.hrldas
! sed -i  '/HRLDAS_SETUP_FILE/c\HRLDAS_SETUP_FILE = "./DOMAIN/wrfinput_d0x.nc"' ./namelist.hrldas
! sed -i  '/START_YEAR/c\START_YEAR = '"$start_year" ./namelist.hrldas
! sed -i  '/START_MONTH/c\START_MONTH = '"$start_month" ./namelist.hrldas
! sed -i  '/START_DAY/c\START_DAY = '"$start_day" ./namelist.hrldas
! sed -i  '/START_HOUR/c\START_HOUR = '"$start_hour" ./namelist.hrldas
! sed -i  '/START_MIN/c\START_MIN = '"$start_minute" ./namelist.hrldas
! sed -i  '/KHOUR =/c\KHOUR = '"$khour" ./namelist.hrldas
! sed -i  '/RESTART_FILENAME_REQUESTED/c\!RESTART_FILENAME_REQUESTED = ""' ./namelist.hrldas
! cat ./namelist.hrldas | grep -E 'HRLDAS_SETUP_FILE|START_|KHOUR'
! mv ./namelist.hrldas {simulation_dir}

In [ ]:
# hydro.namelist  --> RESTART_FILE
! rm -rf hydro.namelist
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/{wrfhydro_version}/trunk/NDHMS/template/HYDRO/hydro.namelist
! sed -i '/GEO_STATIC_FLNM/c\GEO_STATIC_FLNM = "./DOMAIN/geo_em.d0x.nc"' ./hydro.namelist
! sed -i '/RESTART_FILE/c\!RESTART_FILE = ""' ./hydro.namelist
! sed -i '/outlake/c\outlake  = 0' ./hydro.namelist
! sed -i '/output_gw/c\output_gw  = 0' ./hydro.namelist
! sed -i '/GWBASESWCRT/c\GWBASESWCRT  = 0' ./hydro.namelist
! sed -i '/route_lake_f/c\route_lake_f  = ""' ./hydro.namelist
! cat hydro.namelist | grep -E "RESTART|outlake|GWBASESWCRT|route_lake_f"
! mv ./hydro.namelist {simulation_dir}

## Run WRFHydro Model on HPC with CyberGIS Compute

In [ ]:
params_wrfhydro = {"Model_Version": wrfhydro_version,
                   "LSM_Type": "NoahMP",
                   "Forcing_Path": forcing_path_HPC,
                   "Domain_Path": domain_path_HPC,
                   "Merge_Output": "True"}
params_wrfhydro

In [ ]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.create_job_by_ui(defaultJob="wrfhydro-5.x", defaultDataFolder=simulation_dir ,input_params=params_wrfhydro)

## Postprocesing

In [ ]:
!echo {cybergis.recentDownloadPath}/

In [ ]:
!ls -R {cybergis.recentDownloadPath}/

In [ ]:
import xarray as xr
chrtout = xr.open_mfdataset('{}/Outputs/CHRTOUT/*CHRTOUT*'.format(cybergis.recentDownloadPath),
                            combine='by_coords')
chrtout

In [ ]:
chrtout.sel(feature_id =30).streamflow.plot()